In [9]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import SparsePCA

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:3000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainCIR.shape : {trainCIR.shape}')
n_comp = 5
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

def getRealImaginaryParts(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    return real, imag

def apply_sparse_pca(data, n_components):

    # data : (1600, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1) # (1600, 502)
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)
    
    spca = SparsePCA(n_components=n_components, random_state=42, alpha=1)
    data_spca = spca.fit_transform(data_scaled) # (1600, 2)
    
    return data_spca, scaler, spca


# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=10):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep only coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    # Find the index of the smallest residual
    min_residual_index = np.argmin(residuals)
    # Use this index to find the corresponding class
    predicted_class = unique_classes[min_residual_index]
    
    # print(f'predicted_class : {predicted_class}')
    # print(f'residuals : {residuals}')
    return predicted_class

trainCIR.shape : (2400, 15, 251, 2)


IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [1]:
# ----------------- Test -----------------
# Feature Extraction
alice_test_CIRs = testCIR[:, alice_channel, :, :]  # (1600, 251, 2)
eve_test_CIRs = testCIR[:, eve_channel, :, :]      # (1600, 251, 2)

# Compute magnitude for each CIR sample using complex representation
alice_test_magnitude = np.abs(alice_test_CIRs[..., 0] + 1j * alice_test_CIRs[..., 1])  # (1600, 251)
eve_test_magnitude = np.abs(eve_test_CIRs[..., 0] + 1j * eve_test_CIRs[..., 1])        # (1600, 251)
test_cirs = np.vstack((alice_test_magnitude, eve_test_magnitude))  # (3200, 251)

# Apply SparsePCA using the same scaler and SparsePCA fitted on training data
reshaped_test_cirs = test_cirs.reshape(test_cirs.shape[0], -1)  # (samples, 251*2)
test_cirs_scaled = scaler.transform(reshaped_test_cirs)
test_cirs_spca = spca.transform(test_cirs_scaled)  # (samples, n_components)

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_CIRs.shape[0])       # Label '1' for Eve.

# Combine data and labels for testing
test_atoms = test_cirs_spca
test_labels = np.hstack((alice_test_labels, eve_test_labels))


NameError: name 'testCIR' is not defined

In [ ]:
# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)

In [ ]:

accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# Gamma calculation
gamma = (tp + fn) / (tn + fp)

# Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 59.00%

Total testing channel: (400,)
tp: 193
tn: 43
fp: 157
fn: 7
MDR: 0.785
FAR: 0.035
AR: 0.59
